# Семинар: Линейная регрессия

### План семинара
1. Линейная модель
2. Предоработка данных (заполнение пропусков, преобразование нечисловых признаков, масштабирование, генерация новых признаков)
3.  Измерение ошибки в задачах регрессии
4. Обучение линейных моделей 
5. Линейная регрессия и переобучение

### Данные об автомобилях (см. 5_sem)

Для демонстраций загрузим набор данных [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/Automobile). В данных присутствуют категориальные, целочисленные и вещественнозначные признаки (__последняя колонка - стоимость автомобией__).

In [ ]:
import warnings

import matplotlib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

warnings.filterwarnings("ignore")

%matplotlib inline
#plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
X_raw = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
    header=None,
    na_values=["?"],
)

y = X_raw[25]
#матрица "объекты-признаки"
X_raw = X_raw.drop(25, axis=1)

X_raw.head()

In [ ]:
X_raw.shape

## 1. Линейная модель

Будем рассматривать __задачу регрессии__.

Пусть 
$X^l = \{(x_i, y_i)\}_{i=1}^l$ - обучающая выборка,
$y_i$ - значение целевой переменной, $a$ — модель, $a(x_i)$ — прогноз модели на объекте $x_i$. 

$$a(x_i)=  w_0+w_1x_i^1+...+w_dx_i^d$$ 

-- __линейная модель__,
где $w_0, w_1,w_2,...,w_d$ - __веса__ признаков (коэффициенты), $w_0$ - __сдвиг__ (bias). 

Другая форма записи линейной модели:

$$a(x_i)=w_0 + \sum_{j=1}^d w_j x_i^j =  w_0+<w,x_i>, $$ 

где $w=(w_1,w_2,...,w_d)$ - вектор весов, $<w,x_i>$ - скалярное произведение. ).


__В линейной модели предполагается__ конкретный вид зависимости, а именно, что каждый признак линейно
влияет на целевую переменную, и что целевая переменная не зависит от каких-либо
комбинаций признаков.

Вид линейной модели и области ее применимости требуют, чтобы данные были специальным образом подготовлены для того, чтобы линейная модель оказалась адекватной решаемой задаче. 

## 2. Предобработка данных

Предобработка данных:

1. Закодировать категориальные признаки.
2. Обработать NaN.


3. Масштабирование.
4. Убрать линейную зависимость.

### Заполнение пропусков (повторение см. 5_sem)
В матрице объекты-признаки могут быть пропущенные значения, и это вызовет исключение при попытке передать такую матрицу в функцию обучения модели или даже предобработки. Если пропусков немного, можно удалить объекты с пропусками из обучающей выборки. Заполнить пропуски можно разными способами:
* заполнить средними (mean, median);
* предсказывать пропущенные значения по непропущенным.

Часто используют первый вариант - он проще. Для заполнения константами можно использовать метод датафрейма `fillna`, для замены средними — класс `impute.SimpleImputer`.

`Any` возвратит True, если хотя бы один элемент в столбце `True`   

In [ ]:
#узнать: есть или нет пропуски в столбце
#X_raw.isnull().any()

In [ ]:
#число пропусков в каждом столбце
X_raw.isnull().sum()
#any().any()

#### Заполним пропуски средними (вещественнозначные признаки) и пустыми строками (категориальные признаки) (повторение см. 5_sem)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# создаем маску, указывающую на столбцы с категориальными признаками
cat_features_mask = (X_raw.dtypes == "object").values

# для вещественнозначных признаков заполним пропуски средними
X_real = X_raw[X_raw.columns[~cat_features_mask]]

mis_replacer = SimpleImputer(strategy="mean")

X_no_mis_real = pd.DataFrame(
    data=mis_replacer.fit_transform(X_real), columns=X_real.columns
)



In [ ]:
X_no_mis_real.head()

In [ ]:
# для категориальных — пустыми строками
X_cat = X_raw[X_raw.columns[cat_features_mask]].fillna("")


In [ ]:
X_cat.head()

In [ ]:
X_no_mis_real.isnull().sum()

In [ ]:
X_no_mis = pd.concat([X_no_mis_real, X_cat], axis=1)
X_no_mis

In [ ]:
#pd.get_dummies?

### Преобразуем нечисловые признаки при помощи one-hot encoding (повторение см. 5_sem)

In [ ]:
X_dum = pd.get_dummies(X_no_mis, drop_first=True, dtype=float)
print(f"Data shape: {X_dum.shape}")
X_dum.head()

### Масштабирование признаков с помощью MinMaxScaler (повторение см. 5_sem)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
normalizer = MinMaxScaler()
X_real_norm_np = normalizer.fit_transform(X_dum.values)
X = pd.DataFrame(data=X_real_norm_np)
X.head()

### Добавление признаков
__Важным моментом для линейной регрессии__ является нелинейное преобразование признаков. Это позволяет использовать линейную регрессию для моделирования нелинейных зависимостей.

Наиболее популярны такие преобразования:
- добавление полиномиальных признаков (`PolynomialFeatures` в sklearn)
- взятие логарифма признака
- взятие квадратного корня
- применение тригонометрических функций

Небольшой пример. Посмотрев на наши данные, мы можем заметить, что зависимость целевой переменной от шестого признака скорее квадратичная, чем линейная:

In [ ]:
plt.scatter(X[6], y)
plt.show();

In [ ]:
plt.scatter(X[6] ** 2, y)
plt.show();

In [ ]:
#можно так преобразовать
#plt.scatter(X[6], np.sqrt(y))
#plt.show();

А для признака номер 13 линеаризовать зависимость получается с помощью функции $\frac 1 {\sqrt{x}}$:

In [ ]:
plt.scatter(X[13], y)
plt.show();

In [ ]:
plt.scatter(1 / np.sqrt(X[13]), y)
plt.show();

## 3.  Измерение ошибки в задачах регрессии

Итак, $y$ - значение целевой переменной, $a$ — модель. 

$a(x_i)=  w_0+w_1x_i^1+...+w_dx_i^d$
-- __линейная модель__,
где $w_0, w_1,w_2,...,w_d$ -- веса признаков, $l$ - число объектов, $d$ - число признаков.

Чтобы обучать регрессионные модели, нужно определиться, как именно измеряется качество предсказаний.
Важно контролировать качество предсказаний во время обучения, после обучения на новых данных, а также сравнивать качество предсказаний разных двух моделей.

Рассмотрим несколько способов оценить отклонение $L(y, a)$ прогноза от истинного ответа.   
$L(y, a)$ называют <u>функцией потерь</u>, задающей <u>штраф</u> за разницу между предсказанием и истинным значением целевого признака. Свойства функции потерь:
* $L(y_i, a(x_i)) \geqslant 0$;
* $L(y_i, y_i) = 0$.



<u>Функционал качества</u> (<u>функционал ошибки</u>) в задачах обучения с учителем обычно задается в виде суммы по объектам выборки:
$$Q(a) = \frac 1 \ell \sum_{i=1}^\ell L(y_i, a(x_i))$$

__Пример:__ если мы не различаем маленькие ошибки (между 0.01 и 0.1 нет особой разницы) и не хотим получать большие ошибки, можно использовать следующую функцию потерь:

$$L(y_i, a(x_i)) = [| y_i - a(x_i) | < \varepsilon],$$ $\varepsilon$ — допустимая разница между предсказанием и фактом.

### MSE (Mean Squared Error)

$L(y_i, a(x_i)) = (a(x_i) - y_i)^2$

Эта функция наиболее часто используется в задачах регрессии.

__Среднеквадратичная ошибка (Mean Squared Error, MSE)__: 

$$MSE (a, X) = \frac{1}{l}\sum^l_{i=1}(a(x_i) - y_i)^2$$

MSE не обладает свойством устойчивости к __выбросам__ (значение целевой переменной на них либо ошибочно, либо относится к другому распределению и должно быть проигнорировано) потому что задает очень большие штрафы за большие отклонения от фактического значения.

**Пример**. Выберем признак (имеющий индекс 7 в матрице X) и целевой признак (имеющий индекс 15 в матрице X). Добавим к полученной выборке два объекта-выброса и посмотрим, как изменится оптимизированная на MSE прямая.


In [ ]:
from sklearn.linear_model import LinearRegression #про линейную регрессию будет ниже 

In [ ]:
#Будем предсказывать признак 15 по признаку 7
X_subset = X[[7, 15]].values
X_subset[:5]

In [ ]:
#X[15].values

In [ ]:
#добавление двух шумовых точек (двух выбросов)
X_subset_modified = np.vstack((X_subset, [[1, 90], [2, 50]])) 
X_subset_modified[200:]

In [ ]:
#(X[15]==0).sum()
#(X[15]==1).sum()

In [ ]:
#функция для отрисовки данных
def scatter_points_and_plot_line_MSE(
    X_subset: np.array, ax: matplotlib.axes._axes.Axes
) -> None:
    # визуализируем точки (признак - таргет)
    ax.scatter(X_subset[:, 0], X_subset[:, 1])

    # обучим линейную модель (обычная линейная регрессия)
    lr = LinearRegression()
    lr.fit(X_subset[:, 0][:, np.newaxis], X_subset[:, 1])

    # визуализируем прямую
    grid = np.linspace(0, 2, 100) #точки по оси x
    line = lr.predict(grid[:, np.newaxis]) #предсказанные значения 
    ax.plot(grid, line) 
    ax.set_ylim(-20, 100)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].set_title("MSE without outliers") #без выбросов
scatter_points_and_plot_line_MSE(X_subset, ax[0])
ax[1].set_title("MSE with outliers")   #с выбросами
scatter_points_and_plot_line_MSE(X_subset_modified, ax[1])
plt.show();

**Задание.** Реализуйте функцию для подсчета MSE с использованием numpy.

In [ ]:
def MSE(y: np.array, y_pred: np.array) -> np.float64:
    # <YOUR CODE HERE>
    return ((y-y_pred)**2).mean()


a = np.array([11, 20, 19, 17, 10])
pred = np.array([12, 18, 19.5, 18, 9])
mse = MSE(y=a, y_pred=pred)
print(f"Mean Square Error is: {mse}")
assert mse == 1.45

Альтернативный вариант (sklearn):

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print(f"Mean Square Error is: {mean_squared_error(a, pred)}")

In [ ]:
#mean_squared_error?

*__Примечание__. Если использовать MSE как функционал качества (для обучения), то в формуле для MSE на $l$ можно не делить.
Если использовать MSE как метрику качества (для оценки качества модели), то в формуле для MSE нужно деление на $l$, и лучше
извлекать корень из MSE, т.е. иcпользовать метрику RMSE.*

### RMSE (Root Mean Square Error)

Для лучшей интерпретации используется __Root Mean Square Error (RMSE)__: её значение имеет те же масштабы, что и целевая переменная.

$$RMSE (a, X) = \sqrt{MSE (a, X)} = \sqrt{ \sum^l_{i=1}(a(x_i) - y_i)^2}$$

**Задание.** Реализуйте функцию для подсчета RMSE с использованием numpy.

In [ ]:
def RMSE(y: np.array, y_pred: np.array) -> np.float64:
    # <YOUR CODE HERE>
    return np.sqrt(MSE(y,y_pred))

rmse = RMSE(y=a, y_pred=pred)
print(f"Root Mean Square Error is: {rmse}")
assert rmse == 1.2041594578792296

### $R^2$

__Коэффициент детерминации $R^2$__ показывает долю дисперсии в целевой переменной, которая объяснена зависимыми переменными.   ($R^2$ можно интерпретировать как некоторого рода нормированное MSE).

$$R^2(a, X, Y) = 1 - \frac {\sum^l_{i=1}(a(x_i) - y_i)^2}{\sum^L_{i=1}(y_i - \bar{y})^2}$$ или 
$$R^2(a, X, Y) = 1 - \frac {\frac{1}{l}\sum^l_{i=1}(a(x_i) - y_i)^2}{\frac{1}{l}\sum^l_{i=1}(y_i - \bar{y})^2},$$
где 
$\bar{y} = \frac{1}{l}\sum^l_{i=1}y_i$ - среднее значение целевой переменной.
- Если $R^2 < 0$, значит наша модель даёт предсказание хуже константы, то есть абсолютно бесполезна с точки зрения MSE.
- Если $R^2 = 0$, значит мы предсказываем не лучше и не хуже константы в виде среднего значения целевой переменной.
- Если $0 < R^2 < 1$, значит модель работает лучше константного предсказания с точки зрения MSE.
- Если $R^2 = 1$, значит ошибка MSE равна нулю. 

**Задание.** Реализуйте функцию для подсчета $R^2$ с использованием numpy.

In [ ]:
def R_squared(y: np.array, y_pred: np.array) -> np.float64:
    # <YOUR CODE HERE>
    std = ((y - np.mean(y))**2).mean()
    return 1 - MSE(y,y_pred)/std


r_squared = R_squared(y=a, y_pred=pred)
print(f"R2 score is: {r_squared}")

Альтернативный вариант (sklearn):

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print(f"r2_score is: {r2_score(a, pred)}")

### MAE (Mean Absolute Error)

$L(y_i, a(x_i)) = |a(x_i) - y_i|$

__Средняя абсолютная ошибка (Mean Absolute Error, MAE)__:

$$MAE(a, X) = \frac {1}{l} \sum^l_{i=1}|a(x_i) - y_i|$$

В качестве альтернативы MSE можно использовать MAE. Ошибка MAE менее чувствительна к выбросам.

**Задание.** Реализуйте функцию для подсчета MAE с использованием numpy.

In [ ]:
def MAE(y: np.array, y_pred: np.array) -> np.float64:
    # <YOUR CODE HERE>
    return np.abs(y-y_pred).mean()


mae = MAE(y=a, y_pred=pred)
print(f"Mean Absolute Error is: {mae}")
assert mae == 1.1

Альтернативный вариант (sklearn):

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print(f"MAE is: {mean_absolute_error(a, pred)}")

---

### Дополнительно

<u>Можно обучить регрессию, оптимизируя MAE (а не MSE)</u>. В `sklearn` такая регрессия не реализована, но можно использовать модуль `statsmodels` (для работы со статистическими моделями). Более формально, необходимая модель может быть получена с помощью обучения <u>квантильной регрессии</u> с параметром `q=0.5`.

In [ ]:
# !pip install git+https://github.com/statsmodels/statsmodels 
#(если не работает импортирование библиотек ниже)

In [ ]:
import statsmodels.formula.api as smf #синтаксис похож на R

In [ ]:
def scatter_points_and_plot_line_MAE(
    X_subset: np.array, ax: matplotlib.axes._axes.Axes
) -> None:
    # задаем зависимость переменной f15 от переменной f7 и передаем данные
    mod = smf.quantreg("f15 ~ f7", pd.DataFrame(data=X_subset, columns=["f7", "f15"])) #квантильная регрессия 
    res = mod.fit(q=0.5) #q=0.5 - это минимизация MAE
    
    # визуализируем точки
    ax.scatter(X_subset[:, 0], X_subset[:, 1])
 
    # визуализируем прямую
    grid = np.linspace(0, 2, 100)
    line = grid * res.params["f7"] + res.params["Intercept"]
    ax.plot(grid, line)
    ax.set_ylim(-20, 100)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].set_title("MAE without outliers")
scatter_points_and_plot_line_MAE(X_subset, ax[0])
ax[1].set_title("MAE with outliers")
scatter_points_and_plot_line_MAE(X_subset_modified, ax[1])
plt.show();

Прямая практически не изменила направление из-за выбросов! Попробуем добавить больше шумовых объектов.

In [ ]:
#np.random.randint(5, size=60).reshape(-1, 2)* [1, 30][:5]#поэлементное умножение 

In [ ]:
np.random.seed(13)
X_subset_modified_twice = np.vstack(
    (X_subset_modified, np.random.randint(5, size=60).reshape(-1, 2) * [1, 30])
)


In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].set_title("MAE without outliers")
scatter_points_and_plot_line_MAE(X_subset, ax[0])
ax[1].set_title("MAE with outliers")
scatter_points_and_plot_line_MAE(X_subset_modified_twice, ax[1])
plt.show();

При таком количестве выбросов, изменилась даже регрессия над MAE.

---

### Оптимальные константы для MSE и MAE

Допустим, все $l$ объектов выборки имеют одинаковые признаковые описания, но разные значения целевой переменной $y_1,...,y_l$. В этом случае модель должна на всех объектах выдать один и тот же ответ, т.е. алгоритм возвращает константное предсказание: $a(x) = C, C \in R$.  

**Задание.** Найдите $C$, минимизирующий среднеквадратичную ошибку.

**Решение.** Нам необходимо найти константу C, минимизирующую функцию $\frac{1}{n} \sum_{i}^{n} (C - y_i)^2$. Для этого возьмём производную этой функции и приравняем её к нулю: $\frac{2}{n}\sum_{i}^{n} (C - y_i) = 0$. Преобразуем это выражение и выпишем ответ: $C = \frac{\sum_{i}^{n} y_i}{n}$. То есть <u>оптимальная константа</u> — <u>среднее значение целевой переменной</u>.

**Задание.** Найдите $C$, минимизирующий среднюю абсолютную ошибку.


---

## 4. Обучение линейных моделей

Итак, $y$ - значение целевой переменной, $a$ — модель. 

$a(x_i)=  w_0+w_1x_i^1+...+w_dx_i^d$
-- __линейная модель__,
где $w_0, w_1,w_2,...,w_d$ -- веса признаков, $l$ - число объектов, $d$ - число признаков.

__Линейная регрессия__ используется, когда целевая метка линейно зависит от признаков, возможно с каким-то шумом. Нередко линейная регрессия обучается с использованием среднеквадратичной ошибки.
Другое название -  __метод наименьших квадратов, МНК__ - минимизация суммы квадратов разностей ответов $y_i$ и их приближений $a(x_i)$.

Таким образом, получаем __задачу оптимизации__: требуется найти $w_0, w_1,...,w_d$ так, чтобы выполнялось условие:
$$
Q(a,X)=\frac{1}{l}\sum_{i=1}^{l}(y_i-a(x_i))^2
\rightarrow \min_{w_0,w_1,...,w_d}
$$
или (будем считать, что среди признаков есть константный, и поэтому свободный коэффициент можно не писать):
$$
Q(w,X)=
\frac{1}{l}\sum_{i=1}^{l}(<w,x_i>-y_i)^2\rightarrow \min_w
$$
или в матричной форме записи:
$$
Q(w,X)=\frac{1}{l}\|Xw-y\|^2\rightarrow \min_{w}
$$

Дифференцируем $Q$ по параметрам $w_0,w_1,...,w_d$ и приравниваем к нулю, получаем:  
$$ 
\nabla_wQ(w,X)=\frac{2}{l}X^T(Xw-y)=0
$$
Решив уравнение, получим аналитическое решение задачи линейной регрессии:
$$
w^*=(X^TX)^{-1}X^Ty
$$

(_Примечание. Для обучения модели на MSE константа $\frac{1}{l}$ не очень важна, для оценки качества (после обучения) лучше усреднять_).

Реализация линейной регрессии: `sklearn.linear_model.LinearRegression`.   


Частный случай: **парная регрессия**: $a(x)=  w_0+w_1x^1$, 
$$
Q(w_0,w_1,X)=
\sum_{i=1}^{l}(w_1x_i^1+w_0-y_i)^2\rightarrow \min_{w_0,w_1}
$$
$\frac{\partial Q}{\partial w_1}=2\sum_{i=1}^{l}(w_1x_i^1+w_0-y_i)x_i^1$,

$\frac{\partial Q}{\partial w_0}=2\sum_{i=1}^{l}(w_1x_i^1+w_0-y_i)$

Выпишите формулы для $w_1$ и $w_0$.


**Пример 1 ("игрушечный" пример)**. Предсказание линейной моделью (МНК) по трем признакам. (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

$y=3+ 1 \cdot x_1 +2\cdot x_2$ - искомая (приближаемая) функция (вообще говоря, неизвестна). Здесь номер индекса - это номер признака

Дано:

$X=\begin{pmatrix}
  1&1&1\\
  1&1&2\\
  1&2&2\\
  1&2&3\\
\end{pmatrix}$ - матрица "объект-признак" (третий признак = 1 для всех объектов), 
$y=\begin{pmatrix}
  6\\
  8\\
  9\\
  11\\
\end{pmatrix}
$ - ответы

Найти функцию $a(x)$, приближающую $y$: $a(x)=w_0 +w_1\cdot x_1 +w_2\cdot x_2 $

Результат (обучения):

$w=\begin{pmatrix}
  3\\
  1\\
  2\\
\end{pmatrix}
$ - искомые веса,
следовательно, $a(x)=3 +1\cdot x_1 +2\cdot x_2 $

Применение полученной модели к новому объекту $x=(x_1, x_2)$:

$a(x_1=3, x_2=5)=3+1 \cdot 3 +2\cdot 5=16$

**Способ 1 (с помощью LinearRegression())**


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = np.array([[1, 1], 
              [1, 2], 
              [2, 2], 
              [2, 3]]) #признаки 4-х объектов
print('X=',X)
# y = 1 * x_0 + 2 * x_1 + 3 #
y = np.dot(X, np.array([1, 2])) + 3  #целевая переменная
print('y=',y)

lr = LinearRegression() #создаем регрессор
reg = lr.fit(X, y)    #обучение 
print('r2=',reg.score(X, y)) #the coefficient of determination of the prediction
                       #(1 - \frac{u}{v})`
#1.0
print('w1,w2=', reg.coef_)     #искомые параметры w1,w2 
#array([1., 2.])
print('w0=',reg.intercept_) #искомый параметр сдвиг w0
#3.0...
print('a([3, 5])=',reg.predict(np.array([[3, 5]]))) #предсказание для нового объекта x
#array([16.])

In [ ]:
#reg.score?

**Способ 2: формула** $$
w^*=(X^TX)^{-1}X^Ty
$$

In [ ]:
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
print('X=',X)

y = np.dot(X, np.array([1, 2])) + 3
print('y=',y)

X0 = np.array([[1], [1], [1], [1]])
#print('X3=',X3)

#X_mod = np.hstack([X, X3])
X_mod = np.hstack([X0, X])
print('X_mod=',X_mod)

w= np.linalg.inv(X_mod.T@X_mod)@X_mod.T@y #формула w
print('w=',w)

**Пример 2 (парная регрессия)**. Предсказание линейной моделью (МНК) - на наборе данных по диабету. (https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py)

In [ ]:
# Code source: Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)
print(diabetes_X.shape)


In [ ]:
# Use only one feature (столбец №2)
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test, color="black")
plt.plot(diabetes_X_test, diabetes_y_pred, color="blue", linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

## 5. Линейная регрессия и переобучение 
### Данные о сообществах в США



Рассмортрим данные о сообществах в США. 

Предсказывается количество насильственных преступлений относительно численности населения.

[Описание датасета](http://archive.ics.uci.edu/ml/datasets/communities+and+crime)
[Датасет на кэггле](https://www.kaggle.com/kkanda/communities%20and%20crime%20unnormalized%20data%20set)

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### Смотрим на данные (без предобработки)

In [ ]:
data = pd.read_csv("crimedata.csv", na_values=["?"])
data.head()

In [ ]:
data.shape

In [ ]:
# оставим лишь нужные колонки
requiredColumns = [5, 6] + list(range(11, 26)) + list(range(32, 103)) + [145] #номера колонок
data.columns[requiredColumns]  #названия этих колонок в датасете
data = data[data.columns[requiredColumns]] #эти колонки в датасете
data.head()

In [ ]:
data.shape

In [ ]:
# некоторые значения целевой переменной "ViolentCrimesPerPop" пропущены
#выбираем строки, где нет пропусков в "ViolentCrimesPerPop"
data.loc[data["ViolentCrimesPerPop"].notnull(), :].head(3)

In [ ]:
# матрица объект-признак (выбрасываем столбец "ViolentCrimesPerPop")
X = data.loc[data["ViolentCrimesPerPop"].notnull(), :].drop(
    "ViolentCrimesPerPop", axis=1
)


In [ ]:
X.shape

In [ ]:
X.index #индексы строк, которые вошли в X

In [ ]:
#целевая переменная "ViolentCrimesPerPop" с индексами X
y = data["ViolentCrimesPerPop"][X.index] 
y.head()
#y - похож на логнормальное распределение (гистограмма ниже) - можно так оставить 
# np.log1p(y) = log(1 + y) - уже нормальное распределение - или можно применить np.log1p


In [ ]:
y.shape

In [ ]:
plt.hist(y); 
#plt.hist(np.log1p(y)) #можно применить

In [ ]:
#np.log1p?

In [ ]:
#разбиваем на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
#среднее значение (для инф-ии)
y.mean()

### Baseline ("стартовая точка" - без предобработки)
Обучим линейную регрессию и выведем качество по метрике MSE (RMSE) на обучающей и тестовой выборке.

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
lr = LinearRegression() #создаем регрессор
lr.fit(X_train, y_train) #обучение

pred_train = lr.predict(X_train) #предсказание
pred_test = lr.predict(X_test)   #предсказание

print(f"Train: {mean_squared_error(y_train, pred_train)**0.5}") #**0.5
print(f"Test: {mean_squared_error(y_test, pred_test)**0.5}")    #**0.5

r2_score(y_train, pred_train), r2_score(y_test, pred_test)

In [ ]:
#веса модели
plt.hist(lr.coef_);

In [ ]:
#lr.coef_
plt.plot(lr.coef_)

__Признаки переобучения__:
- ошибка на test гораздо больше, чем ошибка на train
- есть большие веса

__Переобучение__ - явление, когда качество построенной модели на новых данных существенно хуже ее качества на обучающей выборке. То есть модель плохо обобщает результаты на новые объекты.

### Регуляризация

Популярным решением для регрессионных моделей является **регуляризация**.

Во время оптимизации линейной регрессии, веса при переменных могут получиться большими в абсолютных значениях. Это не очень хорошо, поскольку модель будет чувствительна к крайне маленьким изменениям в признаках объекта, а значит, переобучена. Для решения проблемы к функционалу ошибки добавляют регуляризатор, который "штрафует" модель за слишком большую норму вектора весов:

$$Q_\alpha(w) = Q(w) + \alpha R(w)$$ 

где $R(w)$ — __регуляризатор__, $\alpha$ - __параметр регуляризации__ ($\alpha \geq 0$).

Наиболее распространенными являются __L1 (Lasso)__ и __L2 (Ridge)__ регуляризаторы:
$$L2: R(w) = ||w||_2^2 = \sum^d_i w_i^2$$

$$L1: R(w) = ||w||_1 = \sum^d_i |w_i|$$

Т.е. функционал ошибки $Q_\alpha(w)$ в каждом из этих случаев будет иметь вид:

$$L2: Q_\alpha(w) = \|Xw-y\|^2\ + \alpha ||w||_2^2$$

$$L1: Q_\alpha(w) = \|Xw-y\|^2\ + \alpha ||w||_1$$

Заметим, что оптимальный вектор весов в случае применения $L2$-регуляризации вместе со среднеквадратичной ошибкой имеет вид:

$$
w^*=(X^TX+\alpha I)^{-1}X^Ty.
$$

Применим каждый из указанных регуляризаторов к нашей задаче и посмотрим на изменение в результатах.


In [ ]:
from sklearn.linear_model import Lasso, Ridge

In [ ]:
lasso = Lasso(5.0).fit(X_train, y_train)
print("Lasso")
print(f"Train: {mean_squared_error(y_train, lasso.predict(X_train))**0.5}")
print(f"Test: {mean_squared_error(y_test, lasso.predict(X_test))**0.5}")

ridge = Ridge(5.0).fit(X_train, y_train)
print("\nRidge")
print(f"Train: {mean_squared_error(y_train, ridge.predict(X_train))**0.5}")
print(f"Test: {mean_squared_error(y_test, ridge.predict(X_test))**0.5}")

__При применении регуляризации всегда нужно масштабирование признаков__.

### Scaling
Попробуем MinMaxScaler.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc = MinMaxScaler()

sc.fit(X_train)

X_train_scaled = pd.DataFrame(data=sc.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=sc.transform(X_test), columns=X_test.columns)

In [ ]:
X_train_scaled.head()

**Задание.** __Обучение линейной регрессии на масштабированных признаках и вывод ошибки на обучающей и тестовой выборках.__

In [ ]:
# <YOUR CODE HERE>

In [ ]:
lr = LinearRegression() #создаем регрессор
lr.fit(X_train_scaled, y_train) #обучение

pred_train = lr.predict(X_train_scaled) #предсказание
pred_test = lr.predict(X_test_scaled)   #предсказание

print(f"Train: {mean_squared_error(y_train, pred_train)**0.5}") #**0.5
print(f"Test: {mean_squared_error(y_test, pred_test)**0.5}")    #**0.5

r2_score(y_train, pred_train), r2_score(y_test, pred_test)

**Задание:** __аналогично с добавлением Ridge регуляризации__

In [ ]:
# <YOUR CODE HERE>

In [ ]:
lasso = Lasso(5.0).fit(X_train_scaled, y_train) #LinearRegression() - 
print("Lasso")
print(f"Train: {mean_squared_error(y_train, lasso.predict(X_train_scaled))**0.5}")
print(f"Test: {mean_squared_error(y_test, lasso.predict(X_test_scaled))**0.5}")

ridge = Ridge(5.0).fit(X_train_scaled, y_train) #Ridge(10.0)
print("\nRidge")
print(f"Train: {mean_squared_error(y_train, ridge.predict(X_train_scaled))**0.5}")
print(f"Test: {mean_squared_error(y_test, ridge.predict(X_test_scaled))**0.5}")
print()
print('Train: {}, {}'.format(r2_score(y_train, lasso.predict(X_train_scaled)), r2_score(y_train, ridge.predict(X_train_scaled))))
print('Test: {}, {}'.format(r2_score(y_test, lasso.predict(X_test_scaled)), r2_score(y_test, ridge.predict(X_test_scaled))))

---

---

### Дополнительно:

#### Отбор признаков на основе дисперсии (high/low variance)

Полезны ли признаки, имеющие высокую дисперсию? А низкую? (и то, и то не очень хорошо) 

In [ ]:
X_train_scaled.var().sort_values()

In [ ]:
#считаем дисперсию (по умолчанию нормируем на N-1, а не на N), сортируем по возрастанию (снизу вверх)
features_variance = X_train_scaled.var().sort_values(ascending=False)
features_variance

Попробуем удалить признаки с самой низкой дисперсией и посмотреть, как изменится качество. В sklearn есть специальный инструмент для такого отбора признаков. Стоит ли нормализовать перед этим признаки?

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# можно убрать все признаки, дисперсия которых меньше заданного значения
vs_transformer = VarianceThreshold(0.01)
vs_transformer.fit(X_train_scaled)
data_train=vs_transformer.transform(X_train_scaled)
data_test=vs_transformer.transform(X_test_scaled)
data_train

In [ ]:
#булева маска выбранных признаков
vs_transformer.get_support()

In [ ]:
X_train_var = pd.DataFrame(
    data=data_train,
    columns=X_train_scaled.columns[vs_transformer.get_support()],
)
X_train_var.shape

In [ ]:
X_test_var = pd.DataFrame(
    data=data_test,
    columns=X_test_scaled.columns[vs_transformer.get_support()],
)

X_test_var.shape

In [ ]:
#Было:
X_train_scaled.shape

In [ ]:
#линейная регрессия
lr = LinearRegression().fit(X_train_var, y_train)
print(f"Train: {mean_squared_error(y_train, lr.predict(X_train_var))**0.5}")
print(f"Test: {mean_squared_error(y_test, lr.predict(X_test_var))**0.5}")

In [ ]:
#L2-регуляризация
ridge = Ridge(5.0).fit(X_train_var, y_train)
print(f"Train: {mean_squared_error(y_train, ridge.predict(X_train_var))**0.5}")
print(f"Test: {mean_squared_error(y_test, ridge.predict(X_test_var))**0.5}")

#### Отбор признаков на основе корреляции с целевой переменной (correlation)
Можно выбрать k признаков, которые дают наиболее высокие значения корреляции с целевой переменной.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
# Выбираем 15 лучших признаков (с т. зр. регрессионной задачи)
sb = SelectKBest(f_regression, k=15)

X_train_kbest = pd.DataFrame(
    data=sb.fit_transform(X_train_var, y_train),
    columns=X_train_var.columns[sb.get_support()],
)
X_test_kbest = pd.DataFrame(
    data=sb.transform(X_test_var), columns=X_test_var.columns[sb.get_support()]
)

In [ ]:
lr = LinearRegression().fit(X_train_kbest, y_train)
print(f"Train: {mean_squared_error(y_train, lr.predict(X_train_kbest))**0.5}")
print(f"Test: {mean_squared_error(y_test, lr.predict(X_test_kbest))**0.5}")

In [ ]:
ridge = Ridge(5.0).fit(X_train_kbest, y_train)
print(f"Train: {mean_squared_error(y_train, ridge.predict(X_train_kbest))**0.5}")
print(f"Test: {mean_squared_error(y_test, ridge.predict(X_test_kbest))**0.5}")

#### Выбор самых значимых признаков с точки зрения регрессии с $L_1$-регуляризацией


In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
#отберем самые лучшие признаки с т.зр. lasso-регрессии
lasso = Lasso(5.0)
l1_select = SelectFromModel(lasso)

X_train_l1 = pd.DataFrame(
    data=l1_select.fit_transform(X_train_var, y_train),
    columns=X_train_var.columns[l1_select.get_support()],
)
X_test_l1 = pd.DataFrame(
    data=l1_select.transform(X_test_var),
    columns=X_test_var.columns[l1_select.get_support()],
)

X_train_l1.shape

In [ ]:
#используем отобранные по lasso признаки в лин. регрессии
lr = LinearRegression().fit(X_train_l1, y_train)
print(f"Train: {mean_squared_error(y_train, lr.predict(X_train_l1))**0.5}")
print(f"Test: {mean_squared_error(y_test, lr.predict(X_test_l1))**0.5}")

#используем отобранные по lasso признаки в лин. регрессии с ridge-регуляризацией
ridge = Ridge(5.0).fit(X_train_l1, y_train)
print(f"Train: {mean_squared_error(y_train, ridge.predict(X_train_l1))**0.5}")
print(f"Test: {mean_squared_error(y_test, ridge.predict(X_test_l1))**0.5}")

#### Зададим все преобразования, отбор признаков и обучение при помощи Pipeline (поток)

In [ ]:
#собираем все шаги (действия, которые мы использовали) вместе
from sklearn.pipeline import Pipeline

pipe = Pipeline(
    steps=[
        ("scaler", MinMaxScaler()),
        ("variance", VarianceThreshold(0.01)),
        ("selection", SelectFromModel(Lasso(5.0))),
        ("regressor", Ridge(5.0)),
    ]
)

pipe.fit(X_train, y_train)

pipe.named_steps

In [ ]:
print(f"Train: {mean_squared_error(y_train, pipe.predict(X_train))**0.5}")
print(f"Test: {mean_squared_error(y_test, pipe.predict(X_test))**0.5}")

#### Можно также настраивать параметры с помощью GridSearch (поиск по сетке):

In [ ]:
pipe.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "variance__threshold": [0.005, 0.0075, 0.009, 0.01, 0.011, 0.012],
    "selection__estimator__alpha": [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0],
    "regressor__alpha": [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0],
}
#сюда помещаем pipe (все действия), param_grid (сетку), делаем кросс-валидацию 
grid_search = GridSearchCV(pipe, param_grid, cv=5)

grid_search.fit(X_train, y_train)

In [ ]:
#лучшие подобранные параметры
pipe_best = grid_search.best_estimator_
pipe_best.named_steps

In [ ]:
#обучение с лучшими параметрами, подобранными по сетке
pipe_best.fit(X_train, y_train)
print(f"Train: {mean_squared_error(y_train, pipe_best.predict(X_train))**0.5}")
print(f"Test: {mean_squared_error(y_test, pipe_best.predict(X_test))**0.5}")